# Coding Club - Prophet
## Forecasting at scale

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nhs-pycom/coding-club/blob/main/introduction-to-prophet/introduction-to-prophet.ipynb)

Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. 

It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

https://opensource.facebook.com/projects

- Forecasting at Scale (https://peerj.com/preprints/3190/)
- https://facebook.github.io/prophet/

Maybe make your own Prophet model from scratch and see the inner workings...
- http://www.degeneratestate.org/posts/2017/Jul/24/making-a-prophet/
- https://www.ritchievink.com/blog/2018/10/09/build-facebooks-prophet-in-pymc3-bayesian-time-series-analyis-with-generalized-additive-models/

Or if you are missing neural networks?
- http://neuralprophet.com/
- https://github.com/ourownstory/neural_prophet

In [ ]:
!pip install -U scikit-learn

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot, plot_cross_validation_metric
from prophet.diagnostics import cross_validation, performance_metrics

from sklearn.metrics import mean_absolute_percentage_error

sns.set_style('whitegrid')

In [ ]:
csv_url = "https://raw.githubusercontent.com/nhs-pycom/coding-club/main/introduction-to-prophet/data/input.csv"

df = pd.read_csv(csv_url, engine="python").dropna()
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y", errors="coerce")

print(df.shape)
df.head()

In [ ]:
df.describe()

In [ ]:
# Let's work with the total of d1 and d2
df["d_total"] = df["d1"] + df["d2"]

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df[["date", "d1", "d2",]].sort_values("date").set_index(
    "date"
).dropna().plot(figsize=(16, 6))

df_plot = (
    df[
        [
            "date",
            "d_total",
        ]
    ]
    .sort_values("date")
    .set_index("date")
    .dropna()
)

df_plot["rolling_7d"] = (
    df[
        [
            "date",
            "d_total",
        ]
    ]
    .sort_values("date")
    .set_index("date")
    .dropna()
    .rolling(window=7, center=True)
    .mean()
)

df_plot.plot(figsize=(16, 6))

# Prophet

In [ ]:
df_prophet = df[["date", "d_total"]].copy(deep=True)
df_prophet.sort_values("date", ascending=True, inplace=True)

df_prophet = df_prophet.dropna().fillna(value=0)

df_prophet.rename(columns={"date": "ds", "d_total": "y"}, inplace=True)
df_prophet.reset_index(drop=True, inplace=True)

# These need to be uncommented for a 'logistic' growth model
# df_prophet['cap'] = 1_000.0
# df_prophet['floor'] = 0.0

print(df_prophet.shape)
df_prophet.tail()

In [ ]:
# Benchmark with a 7-day shifted model
df_benchmark = df_prophet[["ds", "y"]].copy(deep=True)

df_benchmark["y_shift"] = df_benchmark["y"].shift(7)
df_benchmark["y_shift"] = df_benchmark["y_shift"].fillna(method="backfill")

df_benchmark.set_index("ds").plot(figsize=(16, 6))

df_benchmark = df_benchmark.drop("y", axis=1)

In [ ]:
# Benchmark model fit to all data
# Note order of arguments - mean_absolute_percentage_error(y_true, y_pred)
mean_absolute_percentage_error(df_prophet["y"][7:], df_benchmark["y_shift"][7:])

In [ ]:
BH_df = pd.DataFrame(
    {
        "holiday": "bank_hols",
        "ds": pd.to_datetime(
            [
                "2020-12-25",  # Christmas Day
                "2020-12-28",  # Boxing Day BH (shifted)
                "2021-01-01",  # NYD BH
                "2021-04-02",  # Good Friday
                "2021-04-05",  # Easter Monday
            ]
        ),
        "lower_window": -2,
        "upper_window": 2,
    }
)

In [ ]:
%%time

model = Prophet(
    # changepoint_prior_scale=0.5,
    # n_changepoints=25,
    # changepoint_range=0.95,
    # seasonality_mode="multiplicative",
    # seasonality_prior_scale=0.5,
    # holidays=BH_df,
    # holidays_prior_scale=0.5,
    # growth='logistic',
    # mcmc_samples=1000,
)

model.fit(df_prophet)

In [ ]:
fore_len = 7
future = model.make_future_dataframe(periods=fore_len)

# These need to be uncommented for a 'logistic' growth model
# future['cap'] = 1_000.0
# future['floor'] = 0.0

future.tail()

In [ ]:
forecast = model.predict(future)

forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]

In [ ]:
fig = model.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), model, forecast)

In [ ]:
fig2 = model.plot_components(forecast)

In [ ]:
# Prophet model fit to all data
mean_absolute_percentage_error(df_prophet["y"][7:], forecast["yhat"][7:-fore_len])

In [ ]:
df_plot["trend"] = forecast.set_index("ds")["trend"]
df_plot["yhat"] = forecast.set_index("ds")["yhat"]

df_plot_final = df_plot.merge(
    df_benchmark, left_index=True, right_on="ds", how="left"
).set_index("ds")

df_plot_final[["d_total", "rolling_7d", "trend", "yhat", "y_shift"]].plot(
    figsize=(16, 6)
)

Edited from https://facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning


### Parameters that can be tuned or added

`changepoint_prior_scale` **This is probably the most impactful parameter** 

It determines the flexibility of the trend, and in particular how much the trend changes at the trend changepoints. As described in this documentation, if it is too small, the trend will be underfit and variance that should have been modeled with trend changes will instead end up being handled with the noise term. If it is too large, the trend will overfit and in the most extreme case you can end up with the trend capturing yearly seasonality. The default of 0.05 works for many time series, but this could be tuned; a range of [0.001, 0.5] would likely be about right. Parameters like this (regularization penalties; this is effectively a lasso penalty) are often tuned on a log scale.

`seasonality_prior_scale` 

This parameter controls the flexibility of the seasonality. Similarly, a large value allows the seasonality to fit large fluctuations, a small value shrinks the magnitude of the seasonality. The default is 10., which applies basically no regularization. That is because we very rarely see overfitting here (there’s inherent regularization with the fact that it is being modeled with a truncated Fourier series, so it’s essentially low-pass filtered). A reasonable range for tuning it would probably be [0.01, 10]; when set to 0.01 you should find that the magnitude of seasonality is forced to be very small. This likely also makes sense on a log scale, since it is effectively an L2 penalty like in ridge regression.

`holidays`

This is to pass in a dataframe of specified holidays. The holiday effects would be tuned with `holidays_prior_scale`.

`holidays_prior_scale`

This controls flexibility to fit holiday effects. Similar to `seasonality_prior_scale`, it defaults to 10.0 which applies basically no regularization, since we usually have multiple observations of holidays and can do a good job of estimating their effects. This could also be tuned on a range of [0.01, 10] as with seasonality_prior_scale.

`seasonality_mode`

Options are ['additive', 'multiplicative']. Default is 'additive', but many business time series will have multiplicative seasonality. This is best identified just from looking at the time series and seeing if the magnitude of seasonal fluctuations grows with the magnitude of the time series (see the documentation here on multiplicative seasonality), but when that isn’t possible, it could be tuned.

`growth`

Options are ‘linear’ and ‘logistic’. This likely will not be tuned; if there is a known saturating point and growth towards that point it will be included and the logistic trend will be used, otherwise it will be linear.


### Maybe tune?

`changepoint_range`

This is the proportion of the history in which the trend is allowed to change. This defaults to 0.8, 80% of the history, meaning the model will not fit any trend changes in the last 20% of the time series. This is fairly conservative, to avoid overfitting to trend changes at the very end of the time series where there isn’t enough runway left to fit it well. With a human in the loop, this is something that can be identified pretty easily visually: one can pretty clearly see if the forecast is doing a bad job in the last 20%. In a fully-automated setting, it may be beneficial to be less conservative. It likely will not be possible to tune this parameter effectively with cross validation over cutoffs as described above. The ability of the model to generalize from a trend change in the last 10% of the time series will be hard to learn from looking at earlier cutoffs that may not have trend changes in the last 10%. So, this parameter is probably better not tuned, except perhaps over a large number of time series. In that setting, [0.8, 0.95] may be a reasonable range.

### Parameters that would likely not be tuned

`changepoints`

This is for manually specifying the locations of changepoints. None by default, which automatically places them.

`n_changepoints`

This is the number of automatically placed changepoints. The default of 25 should be plenty to capture the trend changes in a typical time series (at least the type that Prophet would work well on anyway). Rather than increasing or decreasing the number of changepoints, it will likely be more effective to focus on increasing or decreasing the flexibility at those trend changes, which is done with `changepoint_prior_scale`.

`yearly_seasonality`

By default (‘auto’) this will turn yearly seasonality on if there is a year of data, and off otherwise. Options are [‘auto’, True, False]. If there is more than a year of data, rather than trying to turn this off during HPO, it will likely be more effective to leave it on and turn down seasonal effects by tuning `seasonality_prior_scale`.

`weekly_seasonality`

Same as for `yearly_seasonality`.

`daily_seasonality`

Same as for `yearly_seasonality`.

`mcmc_samples`

Whether or not MCMC is used will likely be determined by factors like the length of the time series and the importance of parameter uncertainty (these considerations are described in the documentation).

`interval_width`

Prophet predict returns uncertainty intervals for each component, like yhat_lower and yhat_upper for the forecast yhat. These are computed as quantiles of the posterior predictive distribution, and `interval_width` specifies which quantiles to use. The default of 0.8 provides an 80% prediction interval. You could change that to 0.95 if you wanted a 95% interval. This will affect only the uncertainty interval, and will not change the forecast yhat at all and so does not need to be tuned.

`uncertainty_samples`

The uncertainty intervals are computed as quantiles from the posterior predictive interval, and the posterior predictive interval is estimated with Monte Carlo sampling. This parameter is the number of samples to use (defaults to 1000). The running time for predict will be linear in this number. Making it smaller will increase the variance (Monte Carlo error) of the uncertainty interval, and making it larger will reduce that variance. So, if the uncertainty estimates seem jagged this could be increased to further smooth them out, but it likely will not need to be changed. As with `interval_width`, this parameter only affects the uncertainty intervals and changing it will not affect in any way the forecast yhat; it does not need to be tuned.

`stan_backend`

If both `pystan` and `cmdstanpy` backends set up, the backend can be specified. The predictions will be the same, this will not be tuned.

### Diagnostics

In [ ]:
df_cv = cross_validation(model, initial="35 days", period="7 days", horizon="7 days")

In [ ]:
df_cv.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4))

df_cv.groupby("cutoff")["y"].plot(color="midnightblue")
df_cv.groupby("cutoff")["yhat"].plot(color="forestgreen", ls="--")

df_cv.merge(df_benchmark).groupby("cutoff")["y_shift"].plot(color="red", ls="dotted")

plt.show()

In [ ]:
df_perf = performance_metrics(df_cv)

df_perf

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric="mape")

fig.show()

### Next Steps

Our original data consisted of two time series `d1` and `d2` which we summed to model `d_total`.


Instead, try to model the two time series seperately with `Prophet` and see which options work best.


Or if you'd prefer, find a completely different time series to explore (e.g. one which might contain other seasonality components).